In [1]:
import pandas as pd
import numpy as np
import json
import re

In [2]:
def open_files(date):
    focus_text = "2018-02-" + date + "_focus/2018-02-" + \
                 date + "_focus_group.json"
    krx_text = "krx_code.json"
    increase_text = "2018-02-" + date + "_opening_increase.json"
    price_text = "2018-02-" + date + "_price_and_everything.json"
    with open(focus_text, 'r', encoding='UTF-8') as focus_group:
        focus = json.load(focus_group)
    with open(krx_text, 'r', encoding='UTF-8') as krx:
        krx_code = json.load(krx)
    with open(increase_text, 'r', encoding='UTF-8') as oi:
        INCREASE = json.load(oi)
    with open(price_text, 'r', encoding='UTF-8') as oi:
        price = json.load(oi)
    
    return focus, krx_code, INCREASE, price   

In [46]:
def get_time_disc(date):
    
    time = []
    prefix = "2018-02-" + date + "_focus/discussion_2018-02-" + date
    for hour in range(9,16):
        for minute in range(0, 6):
            time.append(prefix + " " + (("0" + str(hour)) if hour <= 9 else str(hour))+ \
                        ":" + str(minute) + "0.json")
    
    time = time[1:40]
    
    return time

In [47]:
def df_list_dis(date):
    df_list = []
    time = get_time_disc(date)
    for x in time:
        with open(x, 'r', encoding='UTF-8') as f:
            discussion = json.load(f)        
            discuss_df = pd.DataFrame(discussion, columns = ["post_num", \
                                "unique_id", "click", "like", "dislike", 
                                "name", "time"])
            reset_col = ["name", "time", "post_num", "unique_id", "click", \
                         "like", "dislike"]
            discuss_df = discuss_df[reset_col]
            df_list.append(discuss_df)
    return df_list
    

In [48]:
def list_to_df(date, df_list, column_names, key_list):
    total = pd.DataFrame(columns=column_names)
    for ind, df in enumerate(df_list):
        if (ind % 3 == 0) and (ind != 0):
            df_total = df.merge(df_list[ind - 3], on = \
                                key_list).merge(df_list[ind - 2], \
                                on = key_list).merge(df_list[ind - 1], on = key_list)
            df_total.columns = column_names
            total = pd.concat([total, df_total], axis = 0)
            total = total[total["name"].isin(INCREASE)]   
    
    return total    

In [49]:
COLUMN_DISC =["name", "time", "post_num", "unique_id", "click", \
              "like", "dislike", "time_1", "post_num_1", "unique_id_1", "click_1", \
              "like_1", "dislike_1", "time_2", "post_num_2", "unique_id_2", "click_2", \
              "like_2", "dislike_2", "time_3", "post_num_3", "unique_id_3", "click_3", \
              "like_3", "dislike_3"]

def get_discuss_df(date):
    discussion_list = df_list_dis(date)
    discuss_df = list_to_df(date, discussion_list, COLUMN_DISC, 
                            ['name']).reset_index().drop(["index"], axis = 1)
    return discuss_df

In [50]:
def df_list_price(date):
    price_df = pd.DataFrame(price, columns = ["index", "code", "name", "time", "price", \
                                         "price_dif", "sell", "buy", "volume", "variation"])
    price_df = price_df[["code", "name", "time", "price", \
                    "price_dif", "sell", "buy", "volume", "variation"]][1:]
    text = "2018-02-" + date + " 09:00"
    price_df = price_df[price_df["time"] != text]
    time_list = price_df['time'].unique().tolist()
    price_df_list = []
    for time in time_list:
        df = price_df[price_df["time"] == time]
        df = df[["code", "name", "time", "price", \
                "price_dif", "sell", "buy", "volume", "variation"]]
        price_df_list.append(df)
        
    return price_df_list

In [51]:
COLUMN_PRICE=["code", "name", "time", "price", "price_dif", "sell", 
              "buy", "volume", "variation", "time_1", "price_1", 
              "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1",
              "time_2", "price_2", "price_dif_2", "sell_2", "buy_2", 
              "volume_2", "variation_2", "time_3", "price_3", 
              "price_dif_3", "sell_3", "buy_3", "volume_3", "variation_3"]

def get_price_df(date):
    price_df_list = df_list_price(date)
    price_df = list_to_df(date, price_df_list, COLUMN_PRICE, \
                          ['code', 'name']).reset_index().drop(["index"], axis = 1)
    
    return price_df

In [52]:
COLUMN_TOTAL = ['name', 'code', 'time', 'price', 'time_1', "price_1", \
                "price_dif_1", "sell_1", "buy_1", "volume_1", "variation_1", \
                'post_num_1', 'unique_id_1', 'click_1', 'like_1', 'dislike_1', \
                'time_2', 'price_2', "price_dif_2", "sell_2", "buy_2", \
                "volume_2", "variation_2", 'post_num_2', 'unique_id_2', 'click_2', \
                'like_2', 'dislike_2', 'time_3', 'price_3', "price_dif_3", \
                "sell_3", "buy_3", "volume_3", "variation_3", 'post_num_3', \
                'unique_id_3', 'click_3', 'like_3', 'dislike_3']

def get_total_df(date):
    price_df = get_price_df(date)
    discuss_df = get_discuss_df(date)
    total_df = pd.merge(price_df, discuss_df, on = ['name', \
                                                    'time', 'time_1', 'time_2', 'time_3'])
    total_df = total_df[COLUMN_TOTAL]
    
    return total_df

In [53]:
import numpy as np
import re

with open("company_size.json", 'r', encoding='UTF-8') as f:
    company_size = json.load(f)

company_df = pd.DataFrame({"company_name": np.zeros(0), \
                           "code": np.zeros(0), "market": np.zeros(0), 
                          "company_size": np.zeros(0)})

company_df = pd.DataFrame(company_size, columns = ["name", "code", "market", "size"])

KOSPI = []
KOSDAQ = []
TRASH = []
MKT_CAP = {}

for idx, row in company_df.iterrows():
    if re.sub('[0-9 ,위]', '', row["market"]) == "코스피":
        KOSPI.append(row["code"])
    elif re.sub('[0-9 ,위]', '', row["market"]) == "코스닥":
        KOSDAQ.append(row["code"])
    else:
        TRASH.append(row["code"])
    
    MKT_CAP[row["code"]] = int(re.sub('[조억원,]', '', row["size"]))

In [54]:
def add_company(date):
    
    total = get_total_df(date)
    total["mkt_cap"] = np.nan
    total["kospi"] = np.nan
    total["kosdaq"] = np.nan
    total["trash"] = np.nan
    
    for index, row in total.iterrows():
        mkt_cap = MKT_CAP[row["code"]]
        if row["code"] in KOSPI:
            kospi = 1
        else:
            kospi = 0

        if row["code"] in KOSDAQ:
            kosdaq = 1
        else:
            kosdaq = 0

        if row["code"] in TRASH:
            trash = 1
        else:
            trash = 0
        total.set_value(index,'mkt_cap', mkt_cap)
        total.set_value(index,'kospi', kospi)
        total.set_value(index,'kosdaq', kosdaq)
        total.set_value(index,'trash', trash)  
    
    return total


In [55]:
def transform_df(date):
    
    total = add_company(date)   
    var_to_transform = ['price', 'price_1', 'price_dif_1', 'sell_1', 
                        'buy_1', 'volume_1', 'variation_1', 'price_2', 
                        'price_dif_2', 'sell_2', 'buy_2', 'volume_2', 
                        'variation_2', 'price_3', 'price_dif_3', 'sell_3', 'buy_3', 
                        'volume_3', 'variation_3']
    for index, row in total.iterrows():
        for var in var_to_transform:
            if isinstance(row[var], str):

                data = row[var].split(",")
                value = ''.join(data)
                value = int(value)
                total.set_value(index, var, value)
    return total
    

In [13]:
focus, krx_code, INCREASE, price = open_files('14')
total_14 = transform_df('14')
focus, krx_code, INCREASE, price = open_files('20')
total_20 = transform_df('20')
focus, krx_code, INCREASE, price = open_files('21')
total_21 = transform_df('21')
focus, krx_code, INCREASE, price = open_files('22')
total_22 = transform_df('22')

In [56]:
focus, krx_code, INCREASE, price = open_files('23')
total_23 = transform_df('23')

In [57]:
total_df = pd.concat([total_14, total_20, total_21, total_22, total_23])

In [58]:
total_df.iloc[100:, :]

,name,code,time,price,time_1,price_1,price_dif_1,sell_1,buy_1,volume_1,...,variation_3,post_num_3,unique_id_3,click_3,like_3,dislike_3,mkt_cap,kospi,kosdaq,trash
100,테라젠이텍스,066700,2018-02-14 10:10,10750,2018-02-14 09:40,10900,300,10900,10850,136566,...,594,13,9,2572,21,16,3735.0,0.0,1.0,0.0
101,이오테크닉스,039030,2018-02-14 10:10,84100,2018-02-14 09:40,84400,400,84500,84400,8340,...,141,2,2,104,0,0,1524.0,0.0,1.0,0.0
102,이화전기,024810,2018-02-14 10:10,364,2018-02-14 09:40,368,0,369,368,4131858,...,2503,24,18,6003,52,48,1325.0,0.0,1.0,0.0
103,한온시스템,018880,2018-02-14 10:10,11950,2018-02-14 09:40,12000,200,12000,11950,76662,...,66,2,2,704,0,3,67793.0,1.0,0.0,0.0
104,신송홀딩스,006880,2018-02-14 10:10,6140,2018-02-14 09:40,6140,90,6150,6050,668,...,1,0,0,0,0,0,736.0,1.0,0.0,0.0
105,락앤락,115390,2018-02-14 10:10,23450,2018-02-14 09:40,23350,50,23350,23300,25104,...,76,1,1,137,2,0,13228.0,1.0,0.0,0.0
106,메디포스트,078160,2018-02-14 10:10,100400,2018-02-14 09:40,101100,2000,101200,101100,41438,...,68,12,8,3334,25,10,8228.0,0.0,1.0,0.0
107,SK디스커버리,006120,2018-02-14 10:10,49100,2018-02-14 09:40,49150,50,49200,49150,7071,...,46,0,0,0,0,0,5203.0,1.0,0.0,0.0
108,테라세미콘,123100,2018-02-14 10:10,25600,2018-02-14 09:40,25850,150,25850,25750,13868,...,249,5,5,666,2,3,3126.0,0.0,1.0,0.0
109,지엠비코리아,013870,2018-02-14 10:10,8700,2018-02-14 09:40,8790,80,8830,8790,41474,...,326,4,4,318,2,2,1791.0,1.0,0.0,0.0


In [59]:
total_df = total_df.reset_index().drop(["index"], axis = 1)

In [60]:
total_df["yesterday_closing_price"] = total_df["price_1"] - total_df["price_dif_1"]

In [61]:
total_df["is_maximum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 > 29.5)
total_df["is_maximum"] = total_df["is_maximum"].astype(int)

In [62]:
total_df["is_minimum"] = (((total_df["price_1"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_2"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5) | \
(((total_df["price_3"] / total_df["yesterday_closing_price"]) - 1) * 100 < -29.5)
total_df["is_minimum"] = total_df["is_maximum"].astype(int)

In [63]:
total_df["price_volatility"] = (((total_df[["price_1", "price_2", "price_3"]].max(axis=1)) / (total_df[["price_1", "price_2", "price_3"]].min(axis=1))) - 1) * 100

In [64]:
total_df["price_trend"] = (((total_df["price_1"] - total_df["price_2"]) < 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) < 0).astype(int)) - (((total_df["price_1"] - total_df["price_2"]) > 0).astype(int) + ((total_df["price_2"] - total_df["price_3"]) > 0).astype(int))

In [65]:
total_df["average_price_volatility"] = total_df["price_trend"] * total_df["price_volatility"] / 2

In [66]:
total_df["sell_minus_buy_1"] = total_df["sell_1"] - total_df["buy_1"]

In [67]:
total_df["sell_minus_buy_2"] = total_df["sell_2"] - total_df["buy_2"]

In [68]:
total_df["sell_minus_buy_3"] = total_df["sell_3"] - total_df["buy_3"]

In [70]:
total_df["is_price_gap_stable"] = ((total_df["sell_minus_buy_1"] == total_df["sell_minus_buy_2"]) & (total_df["sell_minus_buy_2"] == total_df["sell_minus_buy_3"])).astype(int)

In [71]:
total_df["price_gap_volatility"] = (((total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].max(axis=1)) / (total_df[["sell_minus_buy_1", "sell_minus_buy_2", "sell_minus_buy_3"]].min(axis=1)))) - 1

In [72]:
total_df["is_like_higher"] = (total_df["like_3"] > total_df["dislike_3"]).astype(int)

In [73]:
total_df["volume_trend"] = ((((total_df["volume_3"]) - (total_df["volume_2"] * 1+1e-3)) \
 / ((total_df["volume_2"]) - (total_df["volume_1"] * 1-1e-4))) - 1) * 100

In [74]:
total_df["post_num_trend"] = ((((total_df["post_num_3"]) - (total_df["post_num_2"] * 1+1e-3)) \
 / ((total_df["post_num_2"]) - (total_df["post_num_1"] * 1-1e-4))) - 1) * 100

In [75]:
total_df["unique_id_trend"] = ((((total_df["unique_id_3"]) - (total_df["unique_id_2"] * 1+1e-3)) \
 / ((total_df["unique_id_2"]) - (total_df["unique_id_1"] * 1-1e-4))) - 1) * 100

In [76]:
total_df["click_trend"] = ((((total_df["click_3"]) - (total_df["click_2"] * 1+1e-3)) \
 / ((total_df["click_2"]) - (total_df["click_1"] * 1-1e-4))) - 1) * 100

In [78]:
total_df["price_increase"] = ((total_df["price"] / total_df["price_3"]) - 1) * 100

In [79]:
total_df["did_price_increase"] = (total_df["price_increase"] > 0).astype(int)

In [80]:
total_df.to_json('first_data_940_weiwei.json', orient='values')